In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00


In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from transformers import DistilBertModel, DistilBertTokenizer


# Restaurant Review Dataset

In [6]:
# Load your dataset
dataset = pd.read_csv('/content/Restaurant_Reviews.csv')  # Replace with the path to your dataset

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset['Review'], dataset['Liked'], test_size=0.2, random_state=42)

In [7]:
# Initialize the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Tokenize and encode the text data using DistilBERT tokenizer
X_train_encoded = tokenizer.batch_encode_plus(
    X_train.tolist(),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

In [9]:
X_test_encoded = tokenizer.batch_encode_plus(
    X_test.tolist(),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

In [12]:
import torch
# Extract the embeddings from DistilBERT
with torch.no_grad():
    train_embeddings = model(input_ids=X_train_encoded['input_ids'], attention_mask=X_train_encoded['attention_mask'])
    test_embeddings = model(input_ids=X_test_encoded['input_ids'], attention_mask=X_test_encoded['attention_mask'])


In [13]:
# Convert embeddings to numpy arrays
train_embeddings = train_embeddings[0][:, 0, :].numpy()
test_embeddings = test_embeddings[0][:, 0, :].numpy()


In [14]:
# Train an SVM classifier using the DistilBERT embeddings
svm = SVC(kernel='linear')
svm.fit(train_embeddings, y_train)


SVC(kernel='linear')

In [15]:
# Predict sentiment on the test set
y_pred = svm.predict(test_embeddings)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.945054945054945
Recall: 0.8269230769230769
F1-score: 0.882051282051282


In [17]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(train_embeddings, y_train)
lr_pred = lr.predict(test_embeddings)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
# Evaluate the classifiers
lr_accuracy = accuracy_score(y_test, lr_pred)
lr_precision = precision_score(y_test, lr_pred)
lr_recall = recall_score(y_test, lr_pred)
lr_f1 = f1_score(y_test, lr_pred)

In [24]:
# Print results
print("Logistic Regression")
print("Accuracy:", lr_accuracy)
print("Precision:", lr_precision)
print("Recall:", lr_recall)
print("F1-score:", lr_f1)
print()

Logistic Regression
Accuracy: 0.895
Precision: 0.9662921348314607
Recall: 0.8269230769230769
F1-score: 0.8911917098445596



In [25]:
# Random Forest
rf = RandomForestClassifier()
rf.fit(train_embeddings, y_train)
rf_pred = rf.predict(test_embeddings)



In [27]:
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)
rf_f1 = f1_score(y_test, rf_pred)
print("Random Forest")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)

Random Forest
Accuracy: 0.82
Precision: 0.8953488372093024
Recall: 0.7403846153846154
